In [ ]:
#!pip install numba

# Qué es Numba?

De manera muy resumida, Numba una librería para Python que implementa Just In Time Compiling (JIT) e intenta paralelizar código a nivel de CPU.

Funciona bien para librerías matemáticas, como `math` y `numpy`, pero también es posible definir funciones propias que pueden ser paralelizadas.

## Cómo usarla?

Al definir una función, se utiliza el decorador `@njit(parallel=True)` para indicarle a Numba que queremos que compile dicha función y la paralelize para nuestro procesador. La primera vez que se llama a esta función, es compilada y en las llamadas subsiguientes, esta ejecuta directamente desde el blob compilado, con lo que se consigue mejorar el rendimiento al no utilizar el intérprete de Python y además aprovechando de mejor manera los hilos del procesador.

In [14]:
from numba import njit, prange
import numpy as np
import time

x = np.arange(100).reshape(10, 10)

@njit(parallel=True)
def go_fast(a):
    trace = 0.0
    for i in range(a.shape[0]):   # Numba trabaja bien con loops
        trace += np.tanh(a[i, i]) # Numba trabaja bien con Numpy
    return a + trace              # Numba fuciona con type-broadcasting

print(go_fast(x))

[[  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.]
 [ 19.  20.  21.  22.  23.  24.  25.  26.  27.  28.]
 [ 29.  30.  31.  32.  33.  34.  35.  36.  37.  38.]
 [ 39.  40.  41.  42.  43.  44.  45.  46.  47.  48.]
 [ 49.  50.  51.  52.  53.  54.  55.  56.  57.  58.]
 [ 59.  60.  61.  62.  63.  64.  65.  66.  67.  68.]
 [ 69.  70.  71.  72.  73.  74.  75.  76.  77.  78.]
 [ 79.  80.  81.  82.  83.  84.  85.  86.  87.  88.]
 [ 89.  90.  91.  92.  93.  94.  95.  96.  97.  98.]
 [ 99. 100. 101. 102. 103. 104. 105. 106. 107. 108.]]


En la siguiente celda hay tres ejecuciones a las que se les mide el tiempo, la primera corresponde a una función no paralelizada, la sgunda corresponde a la primera ejecución (cuando es compilada) de la función paralelizada, y la tercera corresponde a una ejecución de la función paralelizada ya compilada.

In [15]:
x = np.arange(1000000).reshape(1000, 1000)

def go_slow(a):
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

@njit(parallel=True)
def go_fast(a):
    trace = 0.0
    for i in prange(a.shape[0]):  # prange es equivalente a range, pero indica a numba que puede ser paralelizado
        trace += np.tanh(a[i, i])
    return a + trace

# ejecucion de la funcion sin numba
start = time.time()
go_slow(x)
end = time.time()
print("Tiempo de ejecucion sin numba: ", end - start)

# primera ejecucion y compilacion
start = time.time()
go_fast(x)
end = time.time()
print("Tiempo de ejecucion compilado = %s" % (end - start))

# segunda ejecucion, ya compilada
start = time.time()
go_fast(x)
end = time.time()
print("Tiempo de ejecucion ya compilado = %s" % (end - start))

# el script fue ejecutado en un equipo dual-core, por lo que antes de ejecutar esta celda
# el ouput almacenado debiese mostrar una mejora ligeramente menor a 2x en el tiempo de ejecucion

Tiempo de ejecucion sin numba:  0.014093637466430664
Tiempo de ejecucion compilado = 1.3485193252563477
Tiempo de ejecucion ya compilado = 0.008001089096069336
